In [1]:
import pandas as pd
import numpy as np
import torch

from torch.utils.data import TensorDataset

In [2]:
#!conda install pytorch::pytorch -y

In [3]:
data = pd.read_excel('dataset.xlsx')

In [4]:
data.head()

,청구서번호,No.,Subject,Machinery,Assembly,청구품목,Unnamed: 6,Part No.1,Part No.2,청구량,...,창고입고수량,Control No.,입고창고,창고출고,창고출고수량,출고선박,출고운반선,선박입고,선박입고수량,완료 여부
0,COK-BS-DSP-1901004,1,COK-F-DECK-190104-01,CARGO BOOM VANG BLOCK (STBD 하),BLOCK,"MCKISSICK CONSTRUCTION BLOCKS (WIRE SIZE : 5/8"")",NaN,C15S10BS,2103170,2.0,...,2,19-04-077,BW,2019-05-20,2,COK,ONENESS,2019-06-09,2,완료
1,COK-BS-DSP-1901004,2,COK-F-DECK-190104-01,SPANISH BOOM VANG BLOCK (PORT 상),BLOCK,"MCKISSICK CONSTRUCTION BLOCKS (WIRE SIZE : 5/8"")",NaN,C15D10BS,2103180,1.0,...,1,19-04-064,BW,2019-05-20,1,COK,ONENESS편 탁송,2019-06-09,1,완료
2,COK-BS-DSP-1901004,3,COK-F-DECK-190104-01,PURSE BLOCK,TOW BLOCK,WESTEC 20TON TOW BLOCK,NaN,WESTEC 20TON TOW BLOCK,NaN,1.0,...,1,19-04-077,BW,2019-05-20,1,COK,ONENESS,2019-06-09,1,완료
3,COK-BS-DSP-1905036,1,폰페이 재고정정,MAIN ENGINE,POWER PACK AS,GE POWER PACK FORK - E7(B),NaN,40028340,NaN,8.0,...,8,NaN,PN,2019-08-01,1,DEO,HANARO,2019-08-24,1,완료
4,COK-BS-DSP-1905036,2,폰페이 재고정정,MAIN ENGINE,POWER PACK AS,GE POWER PACK FORK - E7(B),NaN,40028340,NaN,8.0,...,8,NaN,PN,2019-08-29,7,ELS,SL ARCHI,2019-09-30,7,완료


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24664 entries, 0 to 24663
Data columns (total 32 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   청구서번호        24664 non-null  object 
 1   No.          24664 non-null  int64  
 2   Subject      24642 non-null  object 
 3   Machinery    24664 non-null  object 
 4   Assembly     24664 non-null  object 
 5   청구품목         24664 non-null  object 
 6   Unnamed: 6   0 non-null      float64
 7   Part No.1    24640 non-null  object 
 8   Part No.2    3599 non-null   object 
 9   청구량          24546 non-null  float64
 10  견적           24200 non-null  object 
 11  견적수량         24546 non-null  float64
 12  견적화폐         24546 non-null  object 
 13  견적단가         24664 non-null  float64
 14  발주번호         24664 non-null  object 
 15  발주처          24546 non-null  object 
 16  발주           24546 non-null  object 
 17  발주수량         24546 non-null  float64
 18  발주금액         24546 non-null  float64
 19  D/T 

In [6]:
data['Part No.2'].fillna('NaN', inplace=True)

C:\Users\user\AppData\Local\Temp\ipykernel_1484\1059318630.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Part No.2'].fillna('NaN', inplace=True)


In [7]:
data['category']=(
    'Machinery: ' + data['Machinery'] + 
    ', Assembly: ' + data['Assembly'] + 
    ', 청구품목: ' + data['청구품목'] + 
    ', Part No.1: ' + data['Part No.1'] + 
    ', Part No.2: ' + data['Part No.2']
)

In [8]:
print(data['category'].head())

0    Machinery: CARGO BOOM VANG BLOCK (STBD 하), Ass...
1    Machinery: SPANISH BOOM VANG BLOCK (PORT 상), A...
2    Machinery: PURSE BLOCK, Assembly: TOW BLOCK, 청...
3    Machinery: MAIN ENGINE, Assembly: POWER PACK A...
4    Machinery: MAIN ENGINE, Assembly: POWER PACK A...
Name: category, dtype: object


In [9]:
data['category']=data['category'].astype(str)

In [10]:
print(data['category'].apply(type).unique())

[<class 'str'>]


In [11]:
#!conda install anaconda::transformers -y

In [12]:
from transformers import BertTokenizer

C:\Users\user\anaconda3\envs\project\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [16]:
encoded_inputs = tokenizer(
    data['category'].tolist(),
    padding=True,
    truncation=True,
    max_length=128,
    return_tensors='pt'
)

In [17]:
print(encoded_inputs.keys())
#한 데이터=한 행이 {} 안에 딕셔너리처럼 저 세개를 키로 꼭 가지고 있다 => 나중에 임베딩 벡터 추출을 위해 사용할 것임.

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])


## 레이블 잡기 

* 이건 레이블이 없는 데이터고, 사용자가 뭔가를 입력했을 때 그 카테고리에 해당하는 유사품목을 추천해주려면
* 그 카테고리를 정답으로 잡아서 접근해야 할듯

In [18]:
data['combined_label'] = data['Machinery'] + ' | ' + data['Assembly'] #결합하여 사용하겠음.

In [19]:
data['middle_class'] = data['Assembly'].factorize()[0] #레이블 값들을 정수 012345... 로 변환하라.

In [20]:
data['major_class'] = data['Machinery'].factorize()[0]

In [21]:
print(data[['Machinery', 'Assembly', 'combined_label', 'middle_class', 'major_class']].head())

                          Machinery       Assembly  \
0    CARGO BOOM VANG BLOCK (STBD 하)          BLOCK   
1  SPANISH BOOM VANG BLOCK (PORT 상)          BLOCK   
2                       PURSE BLOCK      TOW BLOCK   
3                       MAIN ENGINE  POWER PACK AS   
4                       MAIN ENGINE  POWER PACK AS   

                             combined_label  middle_class  major_class  
0    CARGO BOOM VANG BLOCK (STBD 하) | BLOCK             0            0  
1  SPANISH BOOM VANG BLOCK (PORT 상) | BLOCK             0            1  
2                   PURSE BLOCK | TOW BLOCK             1            2  
3               MAIN ENGINE | POWER PACK AS             2            3  
4               MAIN ENGINE | POWER PACK AS             2            3  


In [23]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments

In [25]:

# 카테고리 텍스트 필드를 토크나이징
encoded_inputs = tokenizer(
    data['category'].tolist(),  # 예: 'Machinery: ' + data['Machinery'] + ' ...'
    padding=True,
    truncation=True,
    max_length=128,
    return_tensors='pt'
)
labels = torch.tensor(data['middle_class'].values)


In [26]:
dataset = TensorDataset(
    encoded_inputs['input_ids'],
    encoded_inputs['attention_mask'],
    labels
)

### BertClassification train

In [27]:
num_labels = len(data['middle_class'].unique())
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_labels)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [28]:
# 학습 인자 설정
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)


### from transformers import Trainer 로 훈련

In [39]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
)

In [40]:
print(type(encoded_inputs['input_ids']), encoded_inputs['input_ids'].shape)


<class 'torch.Tensor'> torch.Size([24664, 101])


In [41]:
print(type(labels), labels.shape)


<class 'torch.Tensor'> torch.Size([24664])


In [42]:
from transformers import DataCollatorWithPadding

# 데이터 콜레이터 설정
data_collator = DataCollatorWithPadding(tokenizer)

# Trainer 설정에 데이터 콜레이터 추가
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    data_collator=data_collator  # 추가된 부분
)

In [43]:
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, input_ids, attention_mask, labels):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_mask[idx],
            'labels': self.labels[idx]
        }

# CustomDataset 인스턴스 생성
dataset = CustomDataset(
    encoded_inputs['input_ids'],
    encoded_inputs['attention_mask'],
    labels
)


In [44]:
print(dataset[0])


{'input_ids': tensor([  101, 10394,  1024,  6636,  8797,  3158,  2290,  3796,  1006,  2358,
         2497,  2094,  1469, 30006,  1007,  1010,  3320,  1024,  3796,  1010,
         1465, 30008, 30025, 29991, 30014, 30004, 30014, 30023, 29995, 30011,
        30020,  1024, 11338, 14270, 19570,  2243,  2810,  5991,  1006,  7318,
         2946,  1024,  1019,  1013,  1022,  1000,  1007,  1010,  2112,  2053,
         1012,  1015,  1024, 27723,  2629,  2015, 10790,  5910,  1010,  2112,
         2053,  1012,  1016,  1024, 12875, 21486, 19841,   102,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 

In [45]:
trainer.train()

Step,Training Loss
10,7.575300
20,7.561300
30,7.573600
40,7.535900
50,7.548700
60,7.539700
70,7.463300
80,7.473600
90,7.498700
100,7.505600


TrainOutput(global_step=9249, training_loss=2.4526858145719683, metrics={'train_runtime': 15989.7805, 'train_samples_per_second': 4.627, 'train_steps_per_second': 0.578, 'total_flos': 3902007616781616.0, 'train_loss': 2.4526858145719683, 'epoch': 3.0})

In [ ]:
# 중분류 레이블이 예측되었을 때, 대분류를 추론
#predicted_middle_class = ...  # 예측된 중분류 레이블
#predicted_major_class = data.loc[data['middle_class'] == predicted_middle_class, 'major_class'].values[0]

### 훈련 / 검증 나눠서 BERTCLASSIFICATION 다시 해보기